### Time Series Analysis with Kats

> [A Python Toolkit by Meta](https://wire.insiderfinance.io/time-series-analysis-with-kats-a-python-toolkit-by-meta-42de2934971d)

Kats stands for "Kit to Analyze Time Series" and it's exactly what it sounds like — a one-stop shop for tackling time series data: a open-source time series analysis toolkit from *Meta*.

Kats wraps up tools for forecasting, detection, feature extraction, and even simulation into a package that's easy to pick up, whether you're a newbie or a seasoned coder. Meta built it for real-world problems, but they’ve kept it approachable.

In [ ]:
!pip install -q -U kats pandas matplotlib

#### Forecasting: Guessing What’s Next

Forecasting is all about peering into the future — like figuring out next quarter’s revenue or tomorrow’s weather. Kats pulls in some heavy hitters to make this happen:

- **ARIMA:** The go-to for spotting trends and seasonal quirks.
- **Prophet:** Meta's own creation, awesome for messy data or holiday spikes.
- **Holt-Winters:** A champ at handling repeating patterns.
- **VAR:** Handy when you’ve got multiple variables playing off each other.

In [ ]:
import pandas as pd
from kats.models.prophet import ProphetModel, ProphetParams
from kats.utils.time_series import TimeSeriesData

# Load dataset
try:
    sales = pd.read_csv("../../data/daily_sales.csv")

    # Ensure dataset contains required columns
    if "time" not in sales.columns or "value" not in sales.columns:
        raise ValueError("Dataset must contain 'time' and 'value' columns.")

    sales["time"] = pd.to_datetime(sales["time"])  # Convert time column to datetime

    # Convert to Kats TimeSeriesData format
    ts_data = TimeSeriesData(sales)

    # Set up Prophet with a seasonal component
    params = ProphetParams(seasonality_mode="multiplicative")
    model = ProphetModel(ts_data, params)

    # Train the model
    model.fit()

    # Forecast the next 30 days
    future = model.predict(steps=30)
    print("Future Forecast:")
    print(future.head())

except FileNotFoundError:
    print("Error: The file 'daily_sales.csv' was not found. Please check the file path.")

except Exception as e:
    print(f"An error occurred: {e}")

#### Detection: Finding the Weird Stuff

In [ ]:
import pandas as pd
from kats.detectors.outlier import OutlierDetector
from kats.utils.time_series import TimeSeriesData

# Load up some traffic numbers
try:
    traffic = pd.read_csv("../../data/web_traffic.csv")
    traffic["time"] = pd.to_datetime(traffic["time"])
    
    # Ensure the dataset has required columns
    if "value" not in traffic.columns:
        raise ValueError("Dataset must contain a 'value' column for analysis.")

    # Convert to Kats TimeSeriesData format
    ts_data = TimeSeriesData(traffic)

    # Fire up the detector
    detector = OutlierDetector(ts_data)
    detector.detector()
    weird_spots = detector.outliers  # List of timestamps where anomalies are detected
    
    print("Strange days:", weird_spots)

except FileNotFoundError:
    print("Error: The file 'web_traffic.csv' was not found. Please check the file path.")

except Exception as e:
    print(f"An error occurred: {e}")

#### Feature Extraction: Digging Out the Good Stuff

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from kats.tsfeatures.tsfeatures import TsFeatures
from kats.utils.time_series import TimeSeriesData

# Load sensor data
try:
    sensors = pd.read_csv("../../data/sensor_logs.csv")  # Ensure you have this file
    sensors["time"] = pd.to_datetime(sensors["time"])  # Convert to datetime
    sensors.set_index("time", inplace=True)  # Set time as index

    # Ensure the dataset has numerical values
    sensor_columns = [col for col in sensors.columns if sensors[col].dtype in ['float64', 'int64']]
    
    if not sensor_columns:
        raise ValueError("No numerical sensor data found. Ensure data contains valid values.")

    # Extract features using Kats
    extractor = TsFeatures()
    ts_data = TimeSeriesData(sensors.reset_index())
    insights = extractor.transform(ts_data)

    # Print Extracted Features
    print("\n📊 Extracted Time-Series Features:")
    print(insights)

    # Visualization: Plot sensor data trends
    plt.figure(figsize=(12, 6))
    for col in sensor_columns:
        plt.plot(sensors.index, sensors[col], label=col)

    plt.title("Sensor Data Trends Over Time")
    plt.xlabel("Time")
    plt.ylabel("Sensor Readings")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.show()

except FileNotFoundError:
    print("❌ Error: The file 'sensor_logs.csv' was not found. Please check the file path.")
except ValueError as e:
    print(f"❌ Error: {e}")
except Exception as e:
    print(f"❌ Unexpected Error: {e}")